In [1]:
!git clone https://github.com/shangjingbo1226/AutoPhrase.git

Cloning into 'AutoPhrase'...
remote: Enumerating objects: 967, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 967 (delta 66), reused 122 (delta 60), pack-reused 830 (from 1)
Receiving objects: 100% (967/967), 199.80 MiB | 10.12 MiB/s, done.
Resolving deltas: 100% (438/438), done.


In [2]:
!cd AutoPhrase/ && ./auto_phrase.sh

===Compilation===
mkdir -p bin
g++ -std=c++11 -Wall -O3 -msse2  -fopenmp  -I.. -pthread -lm -Wno-unused-result -Wno-sign-compare -Wno-unused-variable -Wno-parentheses -Wno-format -o bin/segphrase_train src/main.cpp
g++ -std=c++11 -Wall -O3 -msse2  -fopenmp  -I.. -pthread -lm -Wno-unused-result -Wno-sign-compare -Wno-unused-variable -Wno-parentheses -Wno-format -o bin/segphrase_segment src/segment.cpp
===Downloading Toy Dataset===
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  198M  100  198M    0     0  14.3M      0  0:00:13  0:00:13 --:--:-- 8464k
===Tokenization===

real	4m15.512s
user	7m3.564s
sys	0m16.609s
Detected Language: EN
Current step: Tokenizing wikipedia phrases...
No provided expert labels.
===Part-Of-Speech Tagging===
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent

In [3]:
!cd AutoPhrase/ && ./phrasal_segmentation.sh

===Compilation===
===Tokenization===

real	0m2.603s
user	0m4.425s
sys	0m0.309s
Detected Language: EN
===Part-Of-Speech Tagging===
Current step: Merging...
===Phrasal Segmentation===
=== Current Settings ===
Segmentation Model Path = models/DBLP/segmentation.model
After the phrasal segmentation, only following phrases will be highlighted with <phrase> and </phrase>
	Q(multi-word phrases) >= 0.500000
	Q(single-word phrases) >= 0.800000
POS guided model loaded.
# of loaded patterns = 723100
# of loaded truth patterns = 234490
POS transition matrix loaded
Phrasal segmentation finished.
   # of total highlighted quality phrases = 26460
   # of total processed sentences = 13830
   avg highlights per sentence = 1.91323

real	0m2.041s
user	0m1.837s
sys	0m0.185s
===Generating Output===

real	0m2.356s
user	0m2.777s
sys	0m0.214s


**Did you find any phrases with abnormal scores (e.g. non-phrase with a high score or good phrase with a low score)? Do they show a systematic pattern? What can be the possible reason behind it and how to improve the algorithm to avoid such mistakes**

Some phrases have abnormal scores, like the non-phrase "cs1 cs2" with a high score of 0.97 and the meaningful phrase "surge in" with a low score of 0.10. This likely happens because the model is trained on generalized data, not tuned for specific phrases. To improve, we could fine-tune on domain-specific data, add syntax-based features to detect non-phrases, and use semantic similarity to better assess phrase quality.

In [22]:
import re

# Read the segmented corpus file
with open('segmented_corpus.txt', 'r') as file:
    lines = file.readlines()

# Convert each phrase into a single token
def convert_phrases_to_tokens(lines):
    tokenized_lines = []
    for line in lines:
        # Use regex to find phrases within <phrase>...</phrase> tags
        tokenized_line = re.sub(r'<phrase_Q=[^>]+>([^<]+)</phrase>', lambda m: '_'.join(m.group(1).split()), line)
        tokenized_lines.append(tokenized_line)
    return tokenized_lines

# Save the modified corpus with single tokens
tokenized_corpus = convert_phrases_to_tokens(lines)
with open('tokenized_corpus.txt', 'w') as file:
    for line in tokenized_corpus:
        file.write(line)

In [23]:
from gensim.models import Word2Vec

# Tokenize the tokenized corpus into sentences for Word2Vec
sentences = [line.strip().split() for line in tokenized_corpus]

# Train Word2Vec model on the tokenized corpus
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, workers=4)
w2v_model.save("phrase_embeddings.model")

In [24]:
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
import numpy as np

# Retrieve the phrase embeddings
phrases = list(w2v_model.wv.index_to_key)
embeddings = np.array([w2v_model.wv[phrase] for phrase in phrases])

# KMeans clustering
kmeans = KMeans(n_clusters=6, random_state=0).fit(embeddings)
kmeans_labels = kmeans.labels_

# GMM clustering
gmm = GaussianMixture(n_components=6, random_state=0).fit(embeddings)
gmm_labels = gmm.predict(embeddings)

# Output clusters (20 phrases from each cluster)
for i in range(6):
    kmeans_cluster_phrases = [phrases[j] for j in range(len(phrases)) if kmeans_labels[j] == i][:20]
    gmm_cluster_phrases = [phrases[j] for j in range(len(phrases)) if gmm_labels[j] == i][:20]
    print(f"\nKMeans Cluster {i+1}:", kmeans_cluster_phrases)
    print(f"\nGMM Cluster {i+1}:", gmm_cluster_phrases)



KMeans Cluster 1: ['Algorithms.', 'Techniques.', 'Specification', 'Introduction.', 'Concurrency_Control', 'Internet.', 'Computing', 'deal', 'Artificial_Intelligence', 'Web.', 'Logic_Programming', 'Complexity', 'Use', 'UML.', 'Datenbanksysteme', 'Basic', 'Mining', 'Languages.', 'Visualization', 'Recovery']

GMM Cluster 1: ['Approach.', 'Architecture', 'Algorithms.', 'Techniques.', 'Specification', 'Generation', 'Introduction.', 'Concurrency_Control', 'demonstrated', 'defined', 'Computing', 'deal', 'Artificial_Intelligence', 'Industrial', 'Web.', 'Logic_Programming', 'Mapping', 'Complexity', 'Use', 'time,']

KMeans Cluster 2: ['-', 'Using', 'used', 'paper', 'use', 'present', 'approach', 'Database', 'their', 'models', 'how', 'problem', 'An', 'one', '3-D', 'book', 'set', 'surface', 'well', 'other']

GMM Cluster 2: ['-', 'Using', 'used', 'paper', 'use', 'present', 'approach', 'Database', 'their', 'models', 'how', 'problem', 'An', 'one', '3-D', 'book', 'set', 'surface', 'well', 'other']

KM

**Names of Clusters**

Cluster 1 : Topics in Computer Science

Cluster 2: Misc

Cluster 3: Terms in Information Technology

Cluster 4: Preposition and articles

Cluster 5: Database related terms

Cluster 6: System related terms


---



**Analysis**

GMM can capture more nuanced clusters since it considers that phrases may belong partially to multiple topics (e.g., "Generation," "Mapping," and "time" in Cluster 1 for GMM add interpretative depth).
KMeans, on the other hand, results in more distinct separation, sometimes leading to terms that may feel out of context within the hard boundaries (e.g., "deal" in KMeans Cluster 1).
KMeans assigns each data point strictly to one cluster, giving harder boundaries between clusters.
GMM (Gaussian Mixture Model) allows for probabilistic cluster membership, so phrases can have some likelihood of belonging to multiple clusters, which can lead to more overlap in terms. For instance, in Cluster 1, GMM includes terms like "demonstrated," "defined," and "Industrial," which have a slightly broader scope compared to KMeans.

